In [ ]:
#Necessary imports
!pip install bitsandbytes datasets -U
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
import torch
from datasets import load_dataset, Dataset
from google.colab import drive
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes
import os
from transformers import AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

torch.cuda.empty_cache()
import gc
gc.collect()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

90

In [ ]:
#environment preparation
drive.mount('/content/drive/')
path = '/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi'

#!pip install -q transformers datasets accelerate peft bitsandbytes

#!pip install -U bitsandbytes

model_name = "openai-community/gpt2"

# Define the BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Set the padding token to be the EOS token
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# data preparation

dataset = load_dataset("suayptalha/Poetry-Foundation-Poems")

# Assuming dataset initially only has a 'train' split,
# split the 'train' split into training and testing sets
train_test_split_dataset = dataset['train'].train_test_split(test_size=0.2) # Allocate 20% for testing

def tokenize(example):
    tokenized_output = tokenizer(example["Poem"], padding="max_length", truncation=True, max_length=512)
    # Add labels for causal language modeling training
    tokenized_output["labels"] = tokenized_output["input_ids"]
    return tokenized_output

# Apply tokenization to the new splits
tokenized_dataset = train_test_split_dataset.map(tokenize, batched=True, remove_columns=[ "Title", "Poet", "Tags"])

os.environ["WANDB_DISABLED"] = "true"

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)
config = LoraConfig(r=8, lora_alpha=32, target_modules=["c_attn"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, config)
model.print_trainable_parameters()

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

PoetryFoundationData.csv:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13854 [00:00<?, ? examples/s]

Map:   0%|          | 0/11083 [00:00<?, ? examples/s]

Map:   0%|          | 0/2771 [00:00<?, ? examples/s]

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
#Testing the model before training - seguir una frase
prompt = "The sea is blue, the sky is red"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
output = model.generate(
    **inputs,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The sea is blue, the sky is red
'They are at their best when they don't have to play that game.' The boys were all watching for a sign of what could happen next. 'We're going to get caught up in this nonsense,' said one young man who had just gotten into his early 20s and was heading off on an adventure with three other children – including two older brothers-in‑law - out playing cricket against them as part it's been called from day 1 © Getty Images 2/30 A group of 10 or 12 year old girls walk around walking past each other through bushes near Colchester Common after being forced down by five men during Saturday night action following incidents involving members Of Scotland Yard officers between 5pm & 7am last Friday evening PA 3 / 30 Police officer Peter Gaughan poses outside St James Church Basilica where he took over duty yesterday morning (Picture: AP) Police Officers Pete Gough, Mark Wilson and John Whiteley look about wearing protective gear while patrolling patrol Croydon North

In [ ]:
#Testing the model before training - seguir una instrucció
prompt = "Write a poem about the sea and the sky"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
output = model.generate(
    **inputs,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write a poem about the sea and the sky
I am an old man, my father's name is J.R. Rourke (1811-1966) in English by David Boulton The Sea And Sky By John Aneurin - © 2005 Darryl Zittman Copyright 2002 New York Times Co., Ltd. All rights reserved! This material may not be published under any circumstances without permission from this blog .


In [ ]:

training_args = TrainingArguments(
    output_dir=os.path.join(path, "deep_optimized"),
    per_device_train_batch_size=2,  # Small due to Colab memory limits
    gradient_accumulation_steps=4,  # Simulates effective batch size = 16
    num_train_epochs=1,  # Higher for better generalization
    learning_rate=3e-5,  # Safer than 1e-4; avoids divergence
    weight_decay=0.01,  # Encourages sparsity, reduces overfitting
    lr_scheduler_type="cosine",  # Cosine decay works great for transformers
    warmup_steps=250,  # Gradual ramp-up prevents loss spikes
    fp16=True,  # Great for Colab GPUs
    logging_steps=25,
    save_strategy="epoch",
    save_total_limit=2,
    greater_is_better=False,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)


trainer.train()


<ipython-input-7-6e04c058051d>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
25,7.360100
50,7.200800
75,7.252000
100,7.113300
125,6.933900
150,6.508400
175,6.033500
200,5.306000
225,4.878000
250,4.402000


TrainOutput(global_step=1386, training_loss=3.369143060791544, metrics={'train_runtime': 603.1051, 'train_samples_per_second': 18.377, 'train_steps_per_second': 2.298, 'total_flos': 2905940452442112.0, 'train_loss': 3.369143060791544, 'epoch': 1.0})

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 2.395784616470337, 'eval_runtime': 59.3185, 'eval_samples_per_second': 46.714, 'eval_steps_per_second': 5.85, 'epoch': 1.0}


In [ ]:
#Testing the model before training - seguir una frase
prompt = "The sea is blue, the sky is red"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
output = model.generate(
    **inputs,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The sea is blue, the sky is red, and it smells of fish. I'm going to swim this way for a while."

"Then you'll get some good sleep too," said Lily's voice from behind me—I felt like running through something bad in my stomach instead; "but then what if we go back home?"



In [ ]:
#Testing the model before training - seguir una instrucció
prompt = "Write a poem about the sea and the sky"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
output = model.generate(
    **inputs,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write a poem about the sea and the sky, or tell an anecdote of how fish came out on land with their mouths open.

Or draw up pictures to show us all around us our way home from work: one picture is as beautiful in every direction as another; but it must be remembered that these images are not entirely new at this stage; they have been used by poets before for some time now; those who read them often use old songs which make great music when there comes upon sight something more than usual!


In [ ]:

# Merge LoRA into base model
merged_model = model.merge_and_unload()

# Now merged_model is a standard AutoModelForCausalLM
# You can save it as a normal model
merged_model.save_pretrained(os.path.join(path, "PFP_241_funciona_gpt2"))
tokenizer.save_pretrained(os.path.join(path, "PFP_241_funciona_gpt2"))



('/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi/PFP_241_funciona_gpt2/tokenizer_config.json',
 '/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi/PFP_241_funciona_gpt2/special_tokens_map.json',
 '/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi/PFP_241_funciona_gpt2/vocab.json',
 '/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi/PFP_241_funciona_gpt2/merges.txt',
 '/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi/PFP_241_funciona_gpt2/added_tokens.json',
 '/content/drive/Shareddrives/DeepLearning/UPF_Deep_Learning_2025/Project/Codi/PFP_241_funciona_gpt2/tokenizer.json')